<a href="https://colab.research.google.com/github/bthodla/danano/blob/master/prj4/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gathering data

1.   Load the "twitter-archive-enhanced.csv" into a dataframe
2.   Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library (URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)
3. Using tweet ids from (1) above and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest


In [0]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import io
import tweepy
import json
from google.colab import drive
import yaml
import time
import os


In [0]:
def load_yaml_file(filename: str) -> dict:
    twitter_keys = {}

    with open(filename, 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
    
    twitter_keys['consumer_key'] = cfg['twitter']['consumer']['key']
    twitter_keys['consumer_secret'] = cfg['twitter']['consumer']['secret']
    twitter_keys['access_token'] = cfg['twitter']['access']['token']
    twitter_keys['access_token_secret'] = cfg['twitter']['access']['token-secret']

    return twitter_keys

In [0]:
def print_file(file_name, lines = 10):
    with open(file_name, 'r') as f:
        for _ in range(lines):
            print (f.readline())
        

In [5]:
drive.mount('/content/gdrive')
twitter_keys_yaml_file = '/content/gdrive/My Drive/Colab Notebooks/twitter_keys.yaml'
tweet_json_file = '/content/gdrive/My Drive/Colab Notebooks/tweet_json.txt'
twitter_archive_master_file = '/content/gdrive/My Drive/Colab Notebooks/twitter_archive_master.csv'
dog_stage_file = '/content/gdrive/My Drive/Colab Notebooks/dog_stage.csv'

"""
with open(twitter_keys_yaml_file, encoding='utf-8') as file:
    line = file.readline()
    while line:
        line = file.readline()
        print(line)
"""
print(load_yaml_file(twitter_keys_yaml_file))


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
{'consumer_key': 'Z22dLMPEOtCL2ayqnPTbFg3sK', 'consumer_secret': 'SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj', 'access_token': '14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb', 'access_token_secret': 'nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313'}


In [6]:
# 1. Load the file "twitter-archive-enhanced.csv" into a dataframe
twitter_archive_file_url = 'https://raw.githubusercontent.com/bthodla/danano/master/prj4/twitter-archive-enhanced.csv'

tweets_df = pd.read_csv(twitter_archive_file_url)

tweet_count = tweets_df.shape[0]
print ('%s %d' % ('Tweet Count: ', tweet_count))

Tweet Count:  2356


In [0]:
# 2. Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library

image_predictions_file_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

url_data = requests.get(image_predictions_file_url).content

img_pred_df = pd.read_csv(io.StringIO(url_data.decode('utf-8')), sep = '\t')


In [0]:
""" 3. Using tweet ids from tweet_df and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items 
and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include 
tweet id, retweet count, favorite count and any other data items of interest
"""

twitter_keys = load_yaml_file(twitter_keys_yaml_file)
consumer_key = twitter_keys['consumer_key']
consumer_secret = twitter_keys['consumer_secret']
access_token = twitter_keys['access_token']
access_token_secret = twitter_keys['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

error_tweets = {}
tweet_offset = 0
tweet_batch = 100
tweets_remaining = 0

# Delete the file first so that each run can add content fresh
if os.path.isfile(tweet_json_file):
    os.remove(tweet_json_file)

with open(tweet_json_file, mode = 'a', encoding = 'utf-8') as f:
    while tweet_batch:
        tweet_ids = tweets_df[tweet_offset:tweet_offset + tweet_batch]['tweet_id']
        for tweet_id in tweet_ids:
            try:
                status = api.get_status(tweet_id)
                f.write(json.dumps(status._json) + '\n')

            except tweepy.TweepError as e:
                error_tweets[status.id] = e
            
        tweets_remaining = tweet_count - tweet_offset
        tweet_batch = tweet_batch if tweet_batch < tweets_remaining else tweets_remaining
        tweet_offset += tweet_batch
        # print ('%s %d %s %d' % ('Tweets Remaining: ', tweets_remaining, 'Tweet Offset: ', tweet_offset))
        # time.sleep(1 * 5)


Rate limit reached. Sleeping for: 765
Rate limit reached. Sleeping for: 768


In [0]:
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    print('%s %d %s %d' % ('Valid tweet count: ', len(f.readlines()), 'Error tweet count: ', len(error_tweets)))

In [0]:
# Read "tweet_json.txt" file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest

columns = ['tweet_id', 'text', 'retweet_count', 'favorite_count', 'retweeted', 'favorited', 'media_url']
tweets_mini_df = pd.DataFrame(columns = columns)

index = 0
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    for tweet in f:
        tweet_json = json.loads(tweet)
        tweet_id = tweet_json['id']
        text = tweet_json['text']
        retweet_count = tweet_json['retweet_count']
        favorite_count = tweet_json['favorite_count']
        retweeted = tweet_json['retweeted']
        favorited = tweet_json['favorited']
        try:
            media_url = tweet_json['entities']['media'][0]['media_url_https']
        except KeyError as e:
            media_url = ''

        tweets_mini_df.loc[index] = pd.Series({'tweet_id': tweet_id, 
                                        'text': text, 
                                        'retweet_count': retweet_count, 
                                        'favorite_count': favorite_count,
                                        'retweeted': retweeted,
                                        'favorited': favorited,
                                        'media_url': media_url})
        index +=1


### Assessing data


#### twitter-archive-enhanced.csv

In [9]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [10]:
tweets_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1310,707038192327901184,NaN,NaN,2016-03-08 03:00:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kathmandu. He sees every move you make...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/707038192...,10,10,Kathmandu,None,None,None,None
102,872620804844003328,NaN,NaN,2017-06-08 01:06:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Monkey. She's supporting owners everyw...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/872620804...,13,10,Monkey,None,None,None,None
2321,666435652385423360,NaN,NaN,2015-11-17 02:00:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Can you behave? You're ruining my wedding day...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/666435652...,10,10,None,None,None,None,None
1930,674038233588723717,NaN,NaN,2015-12-08 01:30:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kaiya. She's an aspiring shoe model. 1...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674038233...,12,10,Kaiya,None,None,pupper,None
370,828409743546925057,NaN,NaN,2017-02-06 01:07:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Mutt Ryan. He's quite confident at the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/828409743...,12,10,Mutt,None,None,None,None


In [11]:
tweets_df.shape

(2356, 17)

#### image_predictions.tsv

In [12]:
img_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [13]:
img_pred_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1459,778027034220126208,https://pbs.twimg.com/media/Cswbc2yWcAAVsCJ.jpg,1,clumber,0.946718,True,cocker_spaniel,0.015950,True,Lhasa,0.006519,True
572,678410210315247616,https://pbs.twimg.com/media/CWoyfMiWUAAmGdd.jpg,1,schipperke,0.145877,True,Labrador_retriever,0.098354,True,kelpie,0.097393,True
1070,716285507865542656,https://pbs.twimg.com/media/CfDB3aJXEAAEZNv.jpg,1,Yorkshire_terrier,0.430420,True,silky_terrier,0.196769,True,cairn,0.072676,True
1111,724046343203856385,https://pbs.twimg.com/media/CgxUTS_XEAAC0pv.jpg,1,boxer,0.826272,True,bull_mastiff,0.158595,True,Great_Dane,0.011859,True
896,699691744225525762,https://pbs.twimg.com/media/CbXN7aPWIAE0Xt1.jpg,1,hippopotamus,0.982269,False,sea_lion,0.006295,False,dugong,0.005768,False


In [14]:
img_pred_df.shape

(2075, 12)

#### tweets_mini_df

In [15]:
tweets_mini_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2339 entries, 0 to 2338
Data columns (total 7 columns):
tweet_id          2339 non-null object
text              2339 non-null object
retweet_count     2339 non-null object
favorite_count    2339 non-null object
retweeted         2339 non-null object
favorited         2339 non-null object
media_url         2339 non-null object
dtypes: object(7)
memory usage: 146.2+ KB


In [16]:
tweets_mini_df.sample(5)

,tweet_id,text,retweet_count,favorite_count,retweeted,favorited,media_url
1227,711968124745228288,Meet Winston. He's trapped in a cup of coffee....,2437,8356,False,False,https://pbs.twimg.com/media/CeFrO3qXEAADRbd.jpg
1669,681654059175129088,This is Toffee. He's a happy pupper. Appears d...,983,2736,False,False,https://pbs.twimg.com/media/CXW4wGHWsAE_eBD.jpg
1508,690597161306841088,This is Lolo. She's America af. Behind in scie...,637,2045,False,False,https://pbs.twimg.com/media/CZV-c9NVIAEWtiU.jpg
658,789960241177853952,RT @dog_rates: This is Buddy. His father was a...,4494,0,False,False,
1152,721001180231503872,This is Oliver. Bath time is upon him. His fea...,646,2616,False,False,https://pbs.twimg.com/media/CgGCvxAUkAAx55r.jpg


In [17]:
tweets_mini_df.shape

(2339, 7)

#### Cleaning data

In [0]:
# First, I am making clean copies of each of the dataframes and leaving the originals intact

tweets_df_clean = tweets_df.copy()
img_pred_df_clean = img_pred_df.copy()
tweets_mini_df_clean = tweets_mini_df.copy()

print ('%s %d %s %d' %('Original Row Count: ', tweets_df.shape[0], 'Copy Row Count: ', tweets_df_clean.shape[0]))
print ('%s %d %s %d' %('Original Row Count: ', img_pred_df.shape[0], 'Copy Row Count: ', img_pred_df_clean.shape[0]))
print ('%s %d %s %d' %('Original Row Count: ', tweets_mini_df.shape[0], 'Copy Row Count: ', tweets_mini_df_clean.shape[0]))

Original Row Count:  2356 Copy Row Count:  2356
Original Row Count:  2075 Copy Row Count:  2075
Original Row Count:  2339 Copy Row Count:  2339


#### 1. twitter-archive-enhanced.csv


In [0]:
"""
1.1.1 Define (twitter-archive-enhanced.csv)
There are 181 rows in this dataset that are retweets. I have identified them based non-null values in the "retweeted_status_id" column. 
These need to be removed from the dataset as we only want original tweets
"""

tweets_df_clean[tweets_df_clean['retweeted_status_id'].notnull()].shape[0]

181

In [0]:
"""
1.1.1 Code
"""

tweets_df_clean.drop(tweets_df_clean[tweets_df_clean['retweeted_status_id'].notnull()].index, inplace = True)


In [0]:
"""
1.1.1 Test
Assertion: Original DF Row Count - Cleaned DF Row Count = Retweet Count
"""

assert (tweets_df.shape[0] - tweets_df_clean.shape[0] == tweets_df[tweets_df['retweeted_status_id'].notnull()].shape[0])
# print('%s %d %s %d %s %d' % ('Org. Count: ', tweets_df.shape[0], 'New Count: ', tweets_df_clean.shape[0], 'Diff: ',  tweets_df[tweets_df['retweeted_status_id'].notnull()].shape[0]))

In [0]:
"""
1.1.2 Define (twitter-archive-enhanced.csv)
Remove the following columns from the dataset:

in_reply_to_status_id
in_reply_to_user_id
retweeted_status_id
retweeted_status_user_id
retweeted_status_timestamp
expanded_urls
"""
tweets_df_clean_drop_cols = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls']
tweets_df_clean[tweets_df_clean_drop_cols].head()

,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls
0,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...
1,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...
2,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...
3,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...
4,NaN,NaN,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...


In [0]:
"""
1.1.2 Code
"""
tweets_df_clean.drop(tweets_df_clean_drop_cols, axis = 1, inplace = True)

In [0]:
"""
1.1.2 Test
"""

assert not set(tweets_df_clean_drop_cols).issubset(tweets_df_clean.columns)

In [0]:
""" 
1.1.3 Define (twitter-archive-enhanced.csv)
The "timestamp" column is represented as a string and needs to be converted to datetime
"""

tweets_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null object
source                2175 non-null object
text                  2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: int64(3), object(8)
memory usage: 203.9+ KB


In [0]:
"""
1.1.3 Code
"""

tweets_df_clean['timestamp'] = pd.to_datetime(tweets_df_clean['timestamp'])

In [0]:
"""
1.1.3 Test
"""

tweets_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null datetime64[ns]
source                2175 non-null object
text                  2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 203.9+ KB


In [0]:
"""
1.1.4 Define (twitter-archive-enhanced.csv)
Drop all tweets with timestamps greater than 8/1/2017 as per the project requirements
"""
tweets_df_clean.query('timestamp > "08/01/2017"')

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,13,10,Tilly,None,None,None,None


In [0]:
"""
1.1.4 Code
"""

tweets_df_clean.drop(tweets_df_clean.query('timestamp > "08/01/2017"').index, inplace = True)

In [0]:
"""
1.1.4 Test
"""

assert tweets_df_clean.query('timestamp > "08/01/2017"').shape[0] == 0

In [0]:
"""
1.1.5 & 1.2.1 Define (twitter-archive-enhanced.csv)
Dog stages are represented in separate columns as a sparse-array matrix which is 
wasteful and also not scalable in case we identify new stages. These need to be 
converted to a single column titled "stage". If they are not mutually exclusive 
(meaning, if there are multiple tweets about the same dog at different points in 
time where their stages change), we can put them into a separate dataframe for 
normalization purposes

Dog stages are not fully recorded and many of them contain null values or values labeled as "None". 
We need to scan the tweet text and see if we can extract these values for each row
"""

dog_stages = ['doggo', 'floofer', 'pupper', 'puppo']
[tweets_df_clean[dog_stages] == 'None']

[      doggo  floofer  pupper  puppo
 2      True     True    True   True
 3      True     True    True   True
 4      True     True    True   True
 5      True     True    True   True
 6      True     True    True   True
 7      True     True    True   True
 8      True     True    True   True
 9     False     True    True   True
 10     True     True    True   True
 11     True     True    True   True
 12     True     True    True  False
 13     True     True    True   True
 14     True     True    True  False
 15     True     True    True   True
 16     True     True    True   True
 17     True     True    True   True
 18     True     True    True   True
 20     True     True    True   True
 21     True     True    True   True
 22     True     True    True   True
 23     True     True    True   True
 24     True     True    True   True
 25     True     True    True   True
 26     True     True    True   True
 27     True     True    True   True
 28     True     True    True   True
 

In [0]:
"""
1.1.5 & 1.2.1 Code
In this cleaning exercise, I am going to transpose the "dog_stage columns" to rows first to make the next steps easier
"""

dog_stage_df = pd.melt(tweets_df_clean[['tweet_id', 'text', 'doggo', 'floofer', 'pupper', 'puppo']], id_vars = ['tweet_id', 'text'], var_name = 'dog_stage_col', value_name = 'dog_stage')

# Next, I am assigning the value "doggo" to the column 'dog_stage' in cases where the tweet text has the word "doggo" in it and the 'dog_stage' column has the value "None"
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('doggo ', case = False)].query('dog_stage_col == "doggo" and dog_stage == "None"').index, 'dog_stage'] = 'doggo'

# Next, I am repeating the above for other 'dog_stage' values
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('floofer ', case = False)].query('dog_stage_col == "floofer" and dog_stage == "None"').index, 'dog_stage'] = 'floofer'
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('pupper ', case = False)].query('dog_stage_col == "pupper" and dog_stage == "None"').index, 'dog_stage'] = 'pupper'
dog_stage_df.loc[dog_stage_df[dog_stage_df['text'].str.contains('puppo ', case = False)].query('dog_stage_col == "puppo" and dog_stage == "None"').index, 'dog_stage'] = 'puppo'


In [0]:
"""
1.1.5 & 1.2.1 Code
The test below confirm that there are multiple dog stages tied to a single tweet id.
This seems to indicate that a dog can go through multiple stages.
Therefore, i am going to retain this as a separate dataset
However, I am going to get rid of all rows where the dog stage is None even after fixing the quality
"""
dog_stage_df[['tweet_id', 'dog_stage']].query('dog_stage != "None"').groupby('tweet_id').filter(lambda x: len(x) > 1).sort_values(by=['tweet_id'])

,tweet_id,dog_stage
933,733109485275860992,doggo
5279,733109485275860992,pupper
883,741067306818797568,doggo
5229,741067306818797568,pupper
779,751583847268179968,doggo
5125,751583847268179968,pupper
720,759793422261743616,doggo
5066,759793422261743616,pupper
591,781308096455073793,doggo
4937,781308096455073793,pupper


In [0]:
"""
1.1.5 & 1.2.1 Code
We will begin with dropping the unnecessary columns
"""
dog_stage_df.drop(['text', 'dog_stage_col'], axis = 1, inplace = True)
dog_stage_df.info()

"""
Next, we will drop all rows where the dog_stage column contains the value "None"
"""

dog_stage_df.drop(dog_stage_df.query('dog_stage == "None"').index, inplace = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8692 entries, 0 to 8691
Data columns (total 2 columns):
tweet_id     8692 non-null int64
dog_stage    8692 non-null object
dtypes: int64(1), object(1)
memory usage: 135.9+ KB


In [0]:
"""
1.1.5 & 1.2.1 Test 1
"""

assert dog_stage_df.query('dog_stage == "None"').shape[0] == 0

In [0]:
"""
1.1.5 & 1.2.1 Code
We can now drop the individual dog stage columns from the "tweets_df_clean" dataset
"""
tweets_df_clean.drop(dog_stages, axis = 1, inplace = True)

In [0]:
"""
1.1.5 & 1.2.1 Test 2
"""

assert not set(dog_stages).issubset(tweets_df_clean.columns)

In [0]:
"""
2.1.1 Define (image-predictions.tsv)
In the case of this dataset, we will deal with the tidiness issues first before we deal with data quality issues. 
There is a specific reason for it - there are three breed predictions in this dataset with a percentage of confidence 
against each and a boolean flag indicating whether the image prediction is actually canine or not. 
That makes it a total of 9 columns. I would like to reduce them to three columns to begin with: breed, prediction_confidence, is_canine. 
Addressing this tidiness issue will make it easier to fix the data quality issues identified below. 
We will begin with separating breed predictions to its own dataframe so that we can work on the data quality issues later
"""

img_pred_df_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
483,675432746517426176,https://pbs.twimg.com/media/CV-ef64WoAAbh0I.jpg,1,Labrador_retriever,0.986548,True,golden_retriever,0.008862,True,Chihuahua,0.000694,True
1766,826848821049180160,https://pbs.twimg.com/media/C3mOnZ_XUAAjr2V.jpg,4,Great_Pyrenees,0.858764,True,golden_retriever,0.023526,True,Pekinese,0.017104,True
1798,831650051525054464,https://pbs.twimg.com/media/C4qdThOWAAI3WX3.jpg,1,Eskimo_dog,0.530416,True,Siberian_husky,0.180335,True,Norwegian_elkhound,0.104314,True
692,684222868335505415,https://pbs.twimg.com/media/CX7Y_ByWwAEJdUy.jpg,1,soft-coated_wheaten_terrier,0.791182,True,cocker_spaniel,0.072444,True,teddy,0.071486,False
1471,779377524342161408,https://pbs.twimg.com/ext_tw_video_thumb/77937...,1,sundial,0.170921,False,cash_machine,0.060359,False,maze,0.054981,False


#### 2. image-predictions.tsv


In [0]:
"""
2.1.1 Code
"""

img_pred_df_p1 = img_pred_df_clean[['tweet_id', 'p1', 'p1_conf', 'p1_dog']].copy()
img_pred_df_p1.columns = ['tweet_id', 'breed', 'prediction_confidence', 'is_canine']

img_pred_df_p2 = img_pred_df_clean[['tweet_id', 'p2', 'p2_conf', 'p2_dog']].copy()
img_pred_df_p2.columns = ['tweet_id', 'breed', 'prediction_confidence', 'is_canine']

img_pred_df_p3 = img_pred_df_clean[['tweet_id', 'p3', 'p3_conf', 'p3_dog']].copy()
img_pred_df_p3.columns = ['tweet_id', 'breed', 'prediction_confidence', 'is_canine']

img_pred_df_breed = img_pred_df_p1.append(img_pred_df_p2.append(img_pred_df_p3)).copy()


In [0]:
"""
2.1.1 Test
"""

img_pred_df_breed.sort_values(by = 'tweet_id').head(5)


,tweet_id,breed,prediction_confidence,is_canine
0,666020888022790149,Welsh_springer_spaniel,0.465074,True
0,666020888022790149,Shetland_sheepdog,0.061428,True
0,666020888022790149,collie,0.156665,True
1,666029285002620928,redbone,0.506826,True
1,666029285002620928,Rhodesian_ridgeback,0.072010,True


In [0]:
"""
2.2.1 Define  (image-predictions.tsv)
Once we address the tidiness issue above and spin off the breed predictions to their own dataframe, we will eliminate all predictions where the "is_canine" flag is false. 
We will then pick the prediction with the highest percentage of confidence and retain them as the only breed predictions (assuming that each dog can be classified into a 
single breed) and drop the ones with lower prediction confidence scores

"""
img_pred_df_breed.sort_values(['tweet_id', 'prediction_confidence'], ascending = [True, False]).head(10)

,tweet_id,breed,prediction_confidence,is_canine
0,666020888022790149,Welsh_springer_spaniel,0.465074,True
0,666020888022790149,collie,0.156665,True
0,666020888022790149,Shetland_sheepdog,0.061428,True
1,666029285002620928,redbone,0.506826,True
1,666029285002620928,miniature_pinscher,0.074192,True
1,666029285002620928,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,German_shepherd,0.596461,True
2,666033412701032449,malinois,0.138584,True
2,666033412701032449,bloodhound,0.116197,True
3,666044226329800704,Rhodesian_ridgeback,0.408143,True


In [0]:
"""
2.2.1 Code
"""
img_pred_df_breed = img_pred_df_breed.query('is_canine')
img_pred_df_breed = img_pred_df_breed[img_pred_df_breed.groupby(['tweet_id'])['prediction_confidence'].transform(max) == img_pred_df_breed['prediction_confidence']]


In [0]:
"""
2.2.1 Test
"""
assert img_pred_df_breed.groupby('tweet_id').filter(lambda x: len(x) > 1).shape[0] == 0

In [0]:
"""
2.2.2 Define (image-predictions.tsv)
We can then merge this dataset back to the original dataset

"""
img_pred_df_clean.head(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [0]:
"""
2.2.2 Code
"""

img_pred_df_clean = img_pred_df_clean.merge(img_pred_df_breed, on = 'tweet_id', how = 'left')

In [0]:
"""
2.2.2 Test
"""
# assert set(['breed', 'prediction_confidence', 'is_canine']).issubset(img_pred_df_clean.columns)
list(img_pred_df_clean)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'breed',
 'prediction_confidence',
 'is_canine']

In [0]:
"""
2.2.3 Define (image-predictions.tsv)
After merging, we will drop the following columns as they are no longer needed:

* jpg_url (we have obtained the media_url using the Twitter API and we will retain that in case it needs to be used)
* img_num (don't see how this information will be useful)
* p1
* p1_conf
* p1_dog
* p2
* p2_conf
* p2_dog
* p3
* p3_conf
* p3_dog

"""

img_pred_df_clean_dropped_columns = ['jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'is_canine']

In [0]:
"""
2.2.3 Code
"""
img_pred_df_clean.drop(img_pred_df_clean_dropped_columns, axis = 1, inplace = True)

In [0]:
"""
2.2.3 Test
"""

assert not set(img_pred_df_clean_dropped_columns).issubset(img_pred_df_clean.columns)
img_pred_df_clean.head()

,tweet_id,breed,prediction_confidence
0,666020888022790149,Welsh_springer_spaniel,0.465074
1,666029285002620928,redbone,0.506826
2,666033412701032449,German_shepherd,0.596461
3,666044226329800704,Rhodesian_ridgeback,0.408143
4,666049248165822465,miniature_pinscher,0.560311


#### 3. tweet_mini_df


In [0]:
"""
3.1.1 Define (tweets_mini_df)
We need to drop all the tweets in this dataset that were originally marked as retweets in the "twitter-archive-enhanced" dataset as we are only interested in original tweets
"""

tweets_mini_df_clean = tweets_mini_df.copy()

retweets = tweets_df[tweets_df['retweeted_status_id'].notnull()]['tweet_id'].to_frame(name = 'tweet_id')
retweets.head()

,tweet_id
19,888202515573088257
32,886054160059072513
36,885311592912609280
68,879130579576475649
73,878404777348136964


In [0]:
"""
3.1.1 Code
"""

tweets_mini_df_clean = tweets_mini_df_clean.merge(retweets, on = 'tweet_id', how = 'left', indicator = True)
tweets_mini_df_clean.drop(tweets_mini_df_clean.query('_merge == "both"').index, inplace = True)
tweets_mini_df_clean.drop(['_merge'], axis = 1, inplace = True)


In [0]:
"""
3.1.1 Test
"""

tweets_mini_df_clean.merge(retweets, on = 'tweet_id', how = 'inner')

,tweet_id,text,retweet_count,favorite_count,retweeted,favorited,media_url


In [0]:
"""
3.1.2 Define
We will also drop the "text", "retweeted" and "favorited" columns as we no longer need this (3.1.2)
"""
tweets_mini_df_drop_cols = ['text', 'retweeted', 'favorited']

In [0]:
"""
3.1.2 Code
"""

tweets_mini_df_clean.drop(tweets_mini_df_drop_cols, axis = 1, inplace = True)

In [0]:
"""
3.1.2. Test
"""
assert not set(tweets_mini_df_drop_cols).issubset(tweets_mini_df_clean.columns)

#### 4. Tidiness Issues: combining data from all the datasets


4. Define

Having addressed the data quality and tidiness issues at the level of individual datasets, 
we can now merge the data in these datasets into one. Since the data granularity in each 
of these datasets is at the "tweet_id" level, we will use it to combine the data into a 
single dataset and persist it to a file called "twitter_archive_master.csv". 

We will use "inner joins" to merge these datasets thus eliminating any rows for whcih 
no valid information is available online. For example, there are 17 tweet ids in the 
original "twitter-archive-enhanced" dataset for which the Twitter API returned errors 
and these will be eliminated during the merging process.

Inputs:
 - tweets_df_clean
 - img_pred_df_clean
 - tweets_mini_df_clean

Merged into:
 - twitter_archive_master_df
 
Additional files:
 - Since dog stages seem to have a one-to-many relationship with tweets, they will be 
   stored in their own file


In [0]:
"""
4. Code
"""
twitter_archive_master_df = tweets_df_clean.merge(img_pred_df_clean, on = 'tweet_id', how = 'inner').merge(tweets_mini_df_clean, on = 'tweet_id', how = 'inner')


In [0]:
"""
4. Test
"""

twitter_archive_master_df.head()

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,breed,prediction_confidence,retweet_count,favorite_count,media_url
0,891815181378084864,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,Chihuahua,0.716012,4023,24408,
1,891689557279858688,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,13,10,Darla,Labrador_retriever,0.168086,8369,41060,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg
2,891327558926688256,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,12,10,Franklin,basset,0.555712,9082,39252,
3,891087950875897856,2017-07-29 00:08:17,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,13,10,None,Chesapeake_Bay_retriever,0.425595,3008,19740,
4,890971913173991426,2017-07-28 16:27:12,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,13,10,Jax,Appenzeller,0.341703,1991,11543,


In [0]:
# Write the dataframes to files
twitter_archive_master_df.to_csv(twitter_archive_master_file)


In [0]:
assert os.path.isfile(twitter_archive_master_file)

In [0]:
dog_stage_df.to_csv(dog_stage_file)


In [0]:
assert os.path.isfile(dog_stage_file)